In [1]:
import streamlit as st
import warnings
import configparser
import os
import logging
import random
import string
import time
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from dotenv import load_dotenv


load_dotenv() # Loads .env file

False

In [2]:
from langchain_huggingface import HuggingFaceEmbeddings

In [3]:
GOOGLE_API_KEY = ""
MODEL_EMBEDDINGS_GOOGLE = "models/embedding-001"
MODEL_LLM_GOOGLE = "gemini-1.5-pro"
OUTPUT_PATH = "output"
FAISS_GOOGLE_PATH = "output/faiss_index"
NUMEXPR_MAX_THREADS = "16"
TASK_TYPES = ["task_type_unspecified", "retrieval_query", "retrieval_document", "semantic_similarity", "classification", "clustering"]

In [4]:
CONTEXTUALIZE_Q_SYSTEM_PROMPT = """Given a chat history and the latest user question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is."""

In [5]:
PROMPT_TEMPLATE_GOOGLE = """You are an intelligent assistant. Answer the question based solely on the information provided in the context, specific for Spain.
Do not add any information beyond what is given. 
If the context does not contain the answer, respond with: "The answer is not available in the provided context."
Be concise and accurate. All the answers must be in Spanish language from Spain, in an informal manner.
Answers must be clear, precise, and unambiguous, and a teenager should be able to understand the answer.
Explicitly avoid phrases such as "según el documento", "según el capítulo", "en el texto", "como se menciona en el artículo", or any implication of external texts. Do not construct questions that require knowledge of the structure of the document or the location of information in it.
Include the content-specific information that supports the answer to allow the answer to be independent of any external text.
If the content lacks sufficient information to form a complete answer, do not force one.
Create the answers in your own words; Direct copying of content is not permitted.
NEVER mention the words "documento", "texto", "presentación", "archivo", "tabla", "artículo", "ley", "capítulo", "preámbulo", "título preliminar", "disposición" or "disposiciones generales" in your questions or answers.
ALWAYS make sure that all answers are accurate, self-contained, and relevant, without relying on any original document or text or implying its existence, strictly avoiding any invention or speculation.
IMPORTANT: if in the question there is no mention of a Comunidad Autonoma or the name of a city or province, try that the answer applies to Spain as a country.

Context:
{context}
"""

In [6]:
store = {}

In [7]:
def load_config():
    global GOOGLE_API_KEY
    global MODEL_EMBEDDINGS_GOOGLE
    global OUTPUT_PATH
    global FAISS_GOOGLE_PATH
    global LOG_PATH
    global NUMEXPR_MAX_THREADS
    
    config = configparser.ConfigParser()
    config.read('streamlit_google_history_final.ini')
    GOOGLE_API_KEY = config['KEYS']['google_api_key']
    MODEL_EMBEDDINGS_GOOGLE = config['MODELS']['model_embeddings_google']
    MODEL_LLM_GOOGLE = config['MODELS']['model_llm_google']
    OUTPUT_PATH = config['DEFAULT']['output_path']
    FAISS_GOOGLE_PATH = config['DEFAULT']['faiss_google_path']
    NUMEXPR_MAX_THREADS = config['DEFAULT']['numexpr_max_threads']
    LOG_PATH = config['DEFAULT']['log_path']

In [8]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    global store
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [9]:
def get_conversational_chain(retriever):
    
    llm = ChatGoogleGenerativeAI(model = MODEL_LLM_GOOGLE, temperature = 0.3)
    
    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", CONTEXTUALIZE_Q_SYSTEM_PROMPT),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
    
    history_aware_retriever = create_history_aware_retriever(
        llm, retriever, contextualize_q_prompt
    )
    
    qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", PROMPT_TEMPLATE_GOOGLE),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
    
    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
    
    conversational_rag_chain = RunnableWithMessageHistory(
        rag_chain,
        get_session_history,
        input_messages_key="input",
        history_messages_key="chat_history",
        output_messages_key="answer",
    )
    
    return conversational_rag_chain

In [10]:
def user_input(user_question, k_value, session_id, task_type):
    #embeddings = GoogleGenerativeAIEmbeddings(model = MODEL_EMBEDDINGS_GOOGLE, task_type=task_type)
    embeddings = HuggingFaceEmbeddings(model_name="paraphrase-xlm-r-multilingual-v1")
    #new_db = FAISS.load_local(FAISS_GOOGLE_PATH + "_" + task_type, embeddings, allow_dangerous_deserialization=True)
    new_db = FAISS.load_local("output/faiss_index_ollama", embeddings, allow_dangerous_deserialization=True)
    retriever = new_db.as_retriever(search_kwargs={"k": k_value})
    chain = get_conversational_chain(retriever)
    response = chain.invoke(
        {"input": user_question},
        config={
            "configurable": {"session_id": session_id}
        },
    )
    
    return response

In [11]:
def response_generator(prompt, k_value, session_id):
    start_model_exec = time.time()
    response = user_input(prompt, k_value, session_id)
    end_model_exec = time.time()
    resp_text = "{0} (Tiempo de respuesta: {1:.2f} seg. Session ID: {2}).".format(response, end_model_exec - start_model_exec,  session_id)
    for word in resp_text.split():
        yield word + " "
        time.sleep(0.05)

In [12]:
warnings.filterwarnings("ignore", category=FutureWarning)
load_config()
os.environ["NUMEXPR_MAX_THREADS"] = NUMEXPR_MAX_THREADS
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

In [20]:
k_value = 16
session_id = "hdb0006"
task_type="retrieval_document"
prompt = "Tengo un permiso de conducir de Argentina, como saco el español?"

In [ ]:
response = {}
for task in TASK_TYPES:
    response[task] = user_input(prompt, k_value, session_id, task)

In [ ]:
for task in TASK_TYPES:
    print(task)
    print(response[task]['context'])

In [21]:
response = user_input(prompt, k_value, session_id, task)

In [22]:
response['answer']

'Si tienes un permiso de conducir de Argentina y quieres conducir en España, puedes hacerlo durante los primeros seis meses desde que establezcas tu residencia normal en el país. Después de ese tiempo, tendrás que canjear tu permiso argentino por uno español equivalente. \n'

In [29]:
response = user_input("me lo explicas de otra forma?", k_value, session_id, task)

In [30]:
response['answer']

'Si eres de India y tienes carné de conducir de allí, puedes conducir en España durante seis meses como máximo desde que te establezcas a vivir aquí.  Eso sí, tu carné debe estar en español o con traducción oficial y tienes que tener la edad mínima para conducir en España ese tipo de vehículo. Pasado ese tiempo, tienes que cambiarlo por uno español. \n'

In [31]:
response

{'input': 'me lo explicas de otra forma?',
 'chat_history': [HumanMessage(content='Requisitos para obtener un permiso o una licencia de conducción.'),
  AIMessage(content='Para obtener un permiso o una licencia de conducción necesitas: no tener prohibido conducir por un juez, no tener el carné retirado, haber esperado el tiempo que dice la ley si te quedaste sin puntos, estar física y mentalmente capacitado para conducir, aprobar los exámenes teórico y práctico de la DGT y no tener un permiso de conducir de otro país de la Unión Europea o del Espacio Económico Europeo. \n'),
  HumanMessage(content='Tengo un permiso de conducir de Argentina, como saco el español?'),
  AIMessage(content='Si tienes un permiso de conducir de Argentina y quieres conducir en España, puedes hacerlo durante los primeros seis meses desde que establezcas tu residencia normal en el país. Después de ese tiempo, tendrás que canjear tu permiso argentino por uno español equivalente. \n'),
  HumanMessage(content='y si